In [ ]:
from __future__ import print_function

import gzip
import h5py
import math
import numpy as np
import logging
import os
import os.path as op
import sys

data_path = op.realpath(op.dirname('__file__'))
logger = logging.getLogger('__name__')

In [ ]:
"""
This source code is taken from a higlass/clodius repo.
"""

chrom_col = 1
from_pos_col = 2
to_pos_col = 3
value_col = 5
strand_col = 6
num_rows = 2 # + and -

def bedfile_to_multivec(
    input_filenames,
    f_out,
    bedline_to_chrom_start_end_vector,
    base_resolution,
    has_header,
    chunk_size,
    row_infos=None,
):
    def bedline_to_chrom_start_end_vector(bedlines, row_infos=None):
        chrom_set = set()
        start_set = set()
        end_set = set()
        all_vector = []

        for bedline in bedlines:
            parts = bedline.strip().split()
            chrom = parts[chrom_col - 1]
            start = int(parts[from_pos_col - 1])
            end = int(parts[to_pos_col - 1])
            vector = [
                1 if parts[strand_col - 1] == '+' else np.nan,
                1 if parts[strand_col - 1] == '-' else np.nan,
            ]
            chrom_set.add(chrom)
            start_set.add(start)
            end_set.add(end)

            if len(chrom_set) > 1:
                raise ValueError(
                    "Chromosomes don't match in these lines:", bedlines
                )
            if len(start_set) > 1:
                raise ValueError(
                    "Start positions don't match in these lines:", bedlines
                )
            if len(end_set) > 1:
                raise ValueError(
                    "End positions don't match in these lines:", bedlines
                )
            all_vector += vector

        return (
            list(chrom_set)[0],
            list(start_set)[0],
            list(end_set)[0],
            all_vector,
        )
    """
    Convert an epilogos bedfile to multivec format.
    """
    
    files = []
    for input_filename in input_filenames:
        if op.splitext(input_filename)[1] == ".gz":
            files += [gzip.open(input_filename, "rt")]
        else:
            files += [open(input_filename, "r")]

    FILL_VALUE = np.nan

    # batch regions because h5py is really bad at writing
    batch_length = chunk_size
    batch = []

    # the current index in the dataset
    curr_index = 0
    # the start of the batch in the dataset
    batch_start_index = 0

    if has_header:
        files[0].readline()

    prev_chrom = None
    
    print("base_resolution:", base_resolution)
    for _, lines in enumerate(zip(*files)):

        # Identifies bedfile headers and ignore them
        if lines[0].startswith("browser") or lines[0].startswith("track"):
            continue

        chrom, start, end, vector = bedline_to_chrom_start_end_vector(lines, row_infos)
        print(chrom, start, end, vector)
        # if vector[0] > 0 or vector[1] > 0:
        if len(vector) != len(lines) * num_rows:
            logger.warning("Lines contain fewer columns than expected: %s", lines)
            vector += [np.nan] * (len(lines) * num_rows - len(vector))

        if start % base_resolution != 0:
            logger.error(
                "The start coordinate is not a multiple of the resolution in line: %s",
                lines,
            )
            sys.exit(1)

        if prev_chrom is not None and chrom != prev_chrom:
            # we've reached a new chromosome so we'll dump all
            # the previous values
            # print("len(batch:", len(batch),
            #       "batch_start_index", batch_start_index)
            f_out[prev_chrom][
                batch_start_index : batch_start_index + len(batch)
            ] = np.array(batch)

            # we're starting a new chromosome so we start from the beginning
            curr_index = 0
            batch_start_index = 0
            batch = []

        prev_chrom = chrom

        # print('parts', parts)
        # print('chrom:', chrom, start)

        data_start_index = start // base_resolution

        # if the bedfile skips over a region, we have to add it as empty values
        # to preserve our batch writing
        while curr_index < data_start_index:
            batch += [[FILL_VALUE] * len(vector)]
            curr_index += 1

        """
        if curr_index != data_start_index:
            print("curr_index:", curr_index, data_start_index)
            print("line:", line)
        """

        assert curr_index == data_start_index
        # print('vector', vector)

        # When the binsize is not equal to the base_resolution
        # "break down" the binsize into bins of the rbase_esolution size
        # and add the values to each bin.
        # If there is a remainder, add an additional bin
        data_end_index = math.ceil(end / base_resolution)

        while curr_index < data_end_index:
            batch += [vector]
            curr_index += 1

        # fill in empty

        if len(batch) >= batch_length:
            # dump batch
            try:
                f_out[chrom][
                    batch_start_index : batch_start_index + len(batch)
                ] = np.array(batch)
            except TypeError as ex:
                print("Error:", ex, file=sys.stderr)
                print("Probably need to set the --num-rows parameter", file=sys.stderr)
                return

            batch = []
            batch_start_index = curr_index
            print("dumping batch:", chrom, batch_start_index)

    f_out[chrom][batch_start_index : batch_start_index + len(batch)] = np.array(batch)

In [ ]:
bedfile_to_multivec(
    ["example.bed"],
    'gencode.v18.collapsed.chr1.bed.multivec',
    bedline_to_chrom_start_end_vector,
    1,
    False,
    100000,
)